# Cleaning: Flight Delays

This data set comes from the featured Kaggle datasets, containing three tables that relate to commercial airline flights. The flight delay and cancellation data was collected and published by the DOT's Bureau of Transportation Statistics.

**There are three tables:**

* airlines : contains airline IATA codes with corresponding airline names
* airports : contains
    IATA_CODE:
    Location Identifier
    String

    AIRPORT:
    Airport's Name
    String

    CITY:
    City Name of the Airport
    String

    STATE:
    State Name of the Airport
    String

    COUNTRY:
    Country Name of the Airport
    String

    LATITUDE:
    Latitude of the Airport
    Numeric

    LONGITUDE:
    Longitude of the Airport
    Numeric
* flights : contains many features with descriptions that can be found [here](https://www.kaggle.com/usdot/flight-delays), by selecting flights.csv in the data and clicking on Edit descriptions above the data preview.

## 1. Convert column names to lower case (it's a pain to keep typing upper case column names)

In [1]:
import pandas as pd
import numpy as np

flights = pd.read_csv('flights/flights_sm_raw.csv')
airlines = pd.read_csv('flights/airlines.csv')
airports = pd.read_csv('flights/airports.csv')

In [2]:
f_names = [name.lower() for name in list(flights.columns)]
l_names = [name.lower() for name in list(airlines.columns)]
p_names = [name.lower() for name in list(airports.columns)]

flights.columns = f_names
airlines.columns = l_names
airports.columns = p_names

In [3]:
flights.head()

,year,month,day,day_of_week,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,...,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,2015,1,1,4,US,2013,N584UW,LAX,CLT,30,...,753.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,1674,N853AA,LAS,MIA,35,...,753.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,DL,1560,N3743H,ANC,SEA,45,...,451.0,-24.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,371,N3GXAA,SEA,MIA,100,...,939.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AA,115,N3CTAA,LAX,MIA,105,...,839.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
airlines.head()

,iata_code,airline
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [5]:
airports.head()

,iata_code,airport,city,state,country,latitude,longitude
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


## 2. Remove all rows that have null values for critical features
- critical features are those that are needed to compute missing values
- you need to determine the sets of features that are go together for computing values

In [6]:
flights.iloc[0:5, 9:23]

,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay
0,30,44.0,14.0,13.0,57.0,273.0,249.0,228.0,2125,745.0,8.0,803,753.0,-10.0
1,35,27.0,-8.0,21.0,48.0,268.0,266.0,238.0,2174,746.0,7.0,803,753.0,-10.0
2,45,31.0,-14.0,25.0,56.0,210.0,200.0,171.0,1448,447.0,4.0,515,451.0,-24.0
3,100,52.0,-8.0,30.0,122.0,338.0,347.0,311.0,2724,933.0,6.0,938,939.0,1.0
4,105,103.0,-2.0,14.0,117.0,286.0,276.0,255.0,2342,832.0,7.0,851,839.0,-12.0


1. YEAR: **critical**
1. MONTH: **critical**
1. DAY: **critical**
1. DAY_OF_WEEK: may be derived from year, month and day
1. AIRLINE: **critical**
1. FLIGHT_NUMBER: **critical**
1. TAIL_NUMBER: **critical**
1. ORIGIN_AIRPORT: **critical**
1. DESTINATION_AIRPORT: **critical**
1. SCHEDULED_DEPARTURE: **critical**, timepoint
1. DEPARTURE_TIME = WHEELS_OFF - TAXI_OUT: **critical**, timepoint; we'll derive taxi_out from here
1. DEPARTURE_DELAY = DEPARTURE_TIME-SCHEDULED_DEPARTURE
1. TAXI_OUT: derived from departure_time and wheels_off
1. WHEELS_OFF: **critical**, timepoint
1. SCHEDULED_TIME = SCHEDULED_ARRIVAL-SCHEDULED_DEPARTURE: **critical** cannot calculate because of timezones
1. ELAPSED_TIME = AIR_TIME+TAXI_IN+TAXI_OFF
1. AIR_TIME = WHEELS_OFF - WHEELS_ON: **critical** cannot calculate because of timezones
1. DISTANCE: **critical**
1. WHEELS_ON: **critical**, timepoint
1. TAXI_IN: derived from arrival_time and wheels_on
1. SCHEDULED_ARRIVAL: **critical**, timepoint
1. ARRIVAL_TIME = WHEELS_ON+TAXI_IN: **critical**, timepoint
1. ARRIVAL_DELAY = ARRIVAL_TIME-SCHEDULED_ARRIVAL
1. DIVERTED: **critical**
1. CANCELLED: **critical**
1. CANCELLATION_REASON: **critical**
1. AIR_SYSTEM_DELAY: ?
1. SECURITY_DELAY: ?
1. AIRLINE_DELAY: ?
1. LATE_AIRCRAFT_DELAY: ?
1. WEATHER_DELAY: ?

In [7]:
flights.iloc[:, 9:23].describe()

,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay
count,662804.000000,576651.000000,576651.000000,573757.000000,573757.000000,662798.000000,571395.000000,571395.000000,662804.000000,572641.000000,572641.00000,662804.000000,572641.000000,571395.000000
mean,1334.697796,1335.741712,9.665097,16.055921,1356.680891,139.527372,136.861871,113.388771,802.052239,1470.117538,7.43266,1496.783230,1475.421535,4.404344
std,485.897622,496.812987,37.730697,8.847659,498.284208,74.158283,74.173957,72.182318,598.923492,522.510025,5.62852,509.641339,526.463841,39.078085
min,1.000000,1.000000,-45.000000,1.000000,1.000000,18.000000,15.000000,7.000000,21.000000,1.000000,1.00000,1.000000,1.000000,-82.000000
25%,919.000000,921.000000,-5.000000,11.000000,935.000000,85.000000,82.000000,60.000000,363.000000,1054.000000,4.00000,1109.000000,1058.000000,-13.000000
50%,1330.000000,1330.000000,-2.000000,14.000000,1342.000000,120.000000,118.000000,94.000000,632.000000,1507.000000,6.00000,1523.000000,1511.000000,-5.000000
75%,1735.000000,1740.000000,7.000000,19.000000,1754.000000,170.000000,168.000000,144.000000,1037.000000,1911.000000,9.00000,1922.000000,1916.000000,8.000000
max,2359.000000,2400.000000,1521.000000,185.000000,2400.000000,718.000000,718.000000,669.000000,4983.000000,2400.000000,175.00000,2400.000000,2400.000000,1499.000000


In [8]:
critical = ['scheduled_departure', 'departure_time', 'wheels_off', 'scheduled_time', 'air_time', 'wheels_on', 'scheduled_arrival', 'arrival_time']

for i in range(8):
    flights.drop(flights[flights[critical[i]].isnull()].index, inplace=True)

len(flights)

571395

## 3. Convert times from float values to full 24-hour time format strings

In [9]:
times = ['scheduled_departure', 'departure_time', 'wheels_off', 'wheels_on', 'scheduled_arrival', 'arrival_time']
for i in range(6):
    flights[times[i]] = flights[times[i]].apply(lambda x: ('0000' + str(int(x)))[-4:-2] + ':' + ('0000' + str(int(x)))[-2:] + ':00')

flights[times].head()

,scheduled_departure,departure_time,wheels_off,wheels_on,scheduled_arrival,arrival_time
0,00:30:00,00:44:00,00:57:00,07:45:00,08:03:00,07:53:00
1,00:35:00,00:27:00,00:48:00,07:46:00,08:03:00,07:53:00
2,00:45:00,00:31:00,00:56:00,04:47:00,05:15:00,04:51:00
3,01:00:00,00:52:00,01:22:00,09:33:00,09:38:00,09:39:00
4,01:05:00,01:03:00,01:17:00,08:32:00,08:51:00,08:39:00


## 4. Convert new hour strings to datetime objects
- Test the function and determine the cause of (and fix) any errors

In [16]:
# hour must be in 0..23
for col in times:
    print(col)
    print(flights[flights[col] > '23:59:59'][col].unique())

scheduled_departure
[]
departure_time
['24:00:00']
wheels_off
['24:00:00']
wheels_on
['24:00:00']
scheduled_arrival
[]
arrival_time
['24:00:00']


In [34]:
def convert_dt(year, month, day, time):
        if time == '24:00:00':
            timestamp = pd.to_datetime(str(year) + '-' + str(month) + '-' + str(day) + ' ' + '00:00:00') + pd.Timedelta(1, unit='D')
        else:
            timestamp = pd.to_datetime(str(year) + '-' + str(month) + '-' + str(day) + ' ' + time)
        return timestamp

for i in range(6):
    flights[times[i]] = flights.apply(lambda x: convert_dt(x['year'], x['month'], x['day'], x[times[i]]), axis=1)

In [35]:
flights[times].head()

,scheduled_departure,departure_time,wheels_off,wheels_on,scheduled_arrival,arrival_time
0,2015-01-01 00:30:00,2015-01-01 00:44:00,2015-01-01 00:57:00,2015-01-01 07:45:00,2015-01-01 08:03:00,2015-01-01 07:53:00
1,2015-01-01 00:35:00,2015-01-01 00:27:00,2015-01-01 00:48:00,2015-01-01 07:46:00,2015-01-01 08:03:00,2015-01-01 07:53:00
2,2015-01-01 00:45:00,2015-01-01 00:31:00,2015-01-01 00:56:00,2015-01-01 04:47:00,2015-01-01 05:15:00,2015-01-01 04:51:00
3,2015-01-01 01:00:00,2015-01-01 00:52:00,2015-01-01 01:22:00,2015-01-01 09:33:00,2015-01-01 09:38:00,2015-01-01 09:39:00
4,2015-01-01 01:05:00,2015-01-01 01:03:00,2015-01-01 01:17:00,2015-01-01 08:32:00,2015-01-01 08:51:00,2015-01-01 08:39:00


## 5. Fill in missing values that can be calculated from subsets (our critical features)

In [54]:
calculated = ['departure_delay', 'taxi_out', 'elapsed_time', 'taxi_in', 'arrival_delay']

# departure_delay = departure_time - scheduled_departure
# taxi_out: wheels_off - departure_time
# elapsed_time = air_time + taxi_in + taxi_out
# taxi_in: arrival_time - wheels_on
# arrival_delay = arrival_time - scheduled_arrival

flights[calculated[0]] = (flights['departure_time'] - flights['scheduled_departure']).astype('timedelta64[m]')
flights[calculated[1]] = (flights['wheels_off'] - flights['departure_time']).astype('timedelta64[m]')
flights[calculated[3]] = (flights['arrival_time'] - flights['wheels_on']).astype('timedelta64[m]')
flights[calculated[4]] = (flights['arrival_time'] - flights['scheduled_arrival']).astype('timedelta64[m]')

flights[calculated[2]] = flights['air_time'] + flights['taxi_in'] + flights['taxi_out']

In [55]:
flights[calculated].head()

,departure_delay,taxi_out,elapsed_time,taxi_in,arrival_delay
0,14.0,13.0,249.0,8.0,-10.0
1,-8.0,21.0,266.0,7.0,-10.0
2,-14.0,25.0,200.0,4.0,-24.0
3,-8.0,30.0,347.0,6.0,1.0
4,-2.0,14.0,276.0,7.0,-12.0


In [56]:
flights[calculated].describe()

,departure_delay,taxi_out,elapsed_time,taxi_in,arrival_delay
count,571395.000000,571395.000000,571395.000000,571395.000000,571395.000000
mean,5.193021,12.409776,128.930965,3.132418,-9.874815
std,78.485585,72.643602,126.866389,78.475876,163.432709
min,-1438.000000,-1435.000000,-2797.000000,-1438.000000,-1438.000000
25%,-5.000000,11.000000,82.000000,4.000000,-14.000000
50%,-2.000000,14.000000,117.000000,6.000000,-5.000000
75%,7.000000,19.000000,167.000000,9.000000,7.000000
max,1435.000000,185.000000,718.000000,175.000000,1439.000000


## 6. Filter out rows that contain numeric airport codes
- there is a mix of character IATA codes and numeric strings

In [84]:
flights.reset_index()[['index', 'origin_airport']].head(10)

,index,origin_airport
0,0,LAX
1,1,LAS
2,2,ANC
3,3,SEA
4,4,LAX
5,5,LAS
6,7,SLC
7,9,ANC
8,10,PHX
9,11,SJU


In [87]:
unmatched = pd.merge(flights.reset_index()[['index', 'origin_airport']], airports, left_on='origin_airport', right_on='iata_code', how='left').set_index('index')

flights.drop(unmatched[unmatched['iata_code'].isnull()].index, inplace=True)

In [91]:
unmatched = pd.merge(flights.reset_index()[['index', 'destination_airport']], airports, left_on='destination_airport', right_on='iata_code', how='left').set_index('index')
unmatched[unmatched['iata_code'].isnull()]

,destination_airport,iata_code,airport,city,state,country,latitude,longitude
index,,,,,,,,


In [88]:
len(flights)

522965

In [103]:
flights.query('(cancellation_reason != cancellation_reason) and (air_system_delay != air_system_delay)')

,year,month,day,day_of_week,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,...,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,2015,1,1,4,US,2013,N584UW,LAX,CLT,2015-01-01 00:30:00,...,2015-01-01 07:53:00,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,1674,N853AA,LAS,MIA,2015-01-01 00:35:00,...,2015-01-01 07:53:00,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,DL,1560,N3743H,ANC,SEA,2015-01-01 00:45:00,...,2015-01-01 04:51:00,-24.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,371,N3GXAA,SEA,MIA,2015-01-01 01:00:00,...,2015-01-01 09:39:00,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AA,115,N3CTAA,LAX,MIA,2015-01-01 01:05:00,...,2015-01-01 08:39:00,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,US,413,N571UW,LAS,CLT,2015-01-01 01:20:00,...,2015-01-01 07:54:00,-31.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,1,4,DL,95,N320US,SLC,ATL,2015-01-01 01:40:00,...,2015-01-01 07:25:00,10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,1,4,AS,144,N514AS,ANC,PDX,2015-01-01 02:00:00,...,2015-01-01 06:19:00,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
10,2015,1,1,4,US,699,N172US,PHX,CLT,2015-01-01 02:20:00,...,2015-01-01 07:28:00,-36.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
16,2015,1,1,4,UA,1167,N66837,ORD,DEN,2015-01-01 05:33:00,...,2015-01-01 07:23:00,3.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## 7. BONUS: Write a function and script to correct dates for arrivals that are overnight flights
- don't actually run this code (time consuming)

# Cleaning: Legos

colors : This file contains information on LEGO colors, including a unique ID for each color, its name, and approximate RGB value, and whether it's transparent

inventories : This table contains information on inventories, including a unique ID, it's version and the set number.

inventory_parts : This table contains information part inventories, including a unique ID number, the part number, the color of the part, how many are included and whether it's a spare.

inventory_sets : This file contains information on what inventory is included in which sets, including the inventory ID, the set number and the quantity of that inventory that are included.

part_categories : This dataset includes information on the part category (what type of part it is) and a unique ID for that part category.

parts : This dataset includes information on lego parts, including a unique ID number, the name of the part, and what part category it's from.

sets : This file contains information on LEGO sets, including a unique ID number, the name of the set, the year it was released, its theme and how many parts it includes.

themes : This file includes information on lego themes. Each theme is given a unique ID number, a name, and (if it's part of a bigger theme) which theme it's part of.

## Schema

!['lego data schema'](legos/images/downloads_schema.png)

## 1. Efficiently import all data files into a dictionary for easy access

In [105]:
tables = ['colors', 'inventories', 'inventory_parts', 'inventory_sets', 'part_categories', 'parts', 'sets', 'themes']
legos = {}

for table in tables:
    legos[table] = pd.read_csv('legos/' + table + '.csv')

In [108]:
legos['colors'].head()

,id,name,rgb,is_trans
0,-1,Unknown,0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f
3,2,Green,237841,f
4,3,Dark Turquoise,008F9B,f


## 2. Create a table that provides data that can be used to analyze colors by category
* We do not want detailed part names, part numbers, or category ID
* We also do not want any other ID values
* Make sure the color name and category name columns are labeled clearly

In [148]:
color_cat = pd.merge(legos['colors'], legos['inventory_parts'], left_on='id', right_on='color_id', how='left')
color_cat.drop(['id', 'inventory_id', 'color_id', 'is_spare'], axis=1, inplace=True)
color_cat = pd.merge(color_cat, legos['parts'], left_on='part_num', right_on='part_num', how='outer')
color_cat.drop(['part_num', 'name_y'], axis=1, inplace=True)
color_cat = pd.merge(color_cat, legos['part_categories'], left_on='part_cat_id', right_on='id', how='outer')
color_cat.drop(['part_cat_id', 'id'], axis=1, inplace=True)
color_cat = color_cat.groupby(['name_x', 'rgb', 'is_trans', 'name']).sum().reset_index()
names = ['color_name', 'rgb', 'is_trans', 'category_name', 'number']
color_cat.columns = names
color_cat.sort_values(by='number', ascending=False).head(10)

,color_name,rgb,is_trans,category_name,number
2479,White,FFFFFF,f,Bricks,55812.0
39,Black,05131D,f,Plates,53179.0
56,Black,05131D,f,Technic Pins,51642.0
1773,Red,C91A09,f,Bricks,48251.0
2500,White,FFFFFF,f,Plates,39571.0
2534,Yellow,F2CD37,f,Bricks,38166.0
74,Blue,0055BF,f,Bricks,35878.0
17,Black,05131D,f,Bricks,34969.0
1052,Light Bluish Gray,A0A5A9,f,Plates,31356.0
456,Dark Bluish Gray,6C6E68,f,Plates,29883.0


## 3. Create a table that allows us to analyze set names and their themes
* Merge everything in one command
* Do not include data that will not help with this analysis (remove all of this in a second command)
    + do consider that we might later want to look at specific colors or other part details
* Rename columns where necessary

In [138]:
set_themes = pd.merge(legos['sets'], legos['themes'], left_on='theme_id', right_on='id', how='left')
set_themes.drop(['year', 'theme_id', 'num_parts', 'id'], axis=1, inplace=True)
names = ['set_num', 'set_name', 'theme_name', 'parent_theme_id']
set_themes.columns = names
set_themes

,set_num,set_name,theme_name,parent_theme_id
0,00-1,Weetabix Castle,Castle,411.0
1,0011-2,Town Mini-Figures,Supplemental,67.0
2,0011-3,Castle 2 for 1 Bonus Offer,Lion Knights,186.0
3,0012-1,Space Mini-Figures,Supplemental,126.0
4,0013-1,Space Mini-Figures,Supplemental,126.0
5,0014-1,Space Mini-Figures,Supplemental,126.0
6,0015-1,Space Mini-Figures,Supplemental,126.0
7,0016-1,Castle Mini Figures,Castle,NaN
8,00-2,Weetabix Promotional House 1,Building,411.0
9,00-3,Weetabix Promotional House 2,Building,411.0


## 4. Create a copy of the parts table and modify it to show what sets the parts belong to

In [145]:
parts_copy = legos['parts'].copy()
parts_copy = pd.merge(parts_copy, legos['inventory_parts'], left_on='part_num', right_on='part_num', how='left')
parts_copy = pd.merge(parts_copy, legos['inventories'], left_on='inventory_id', right_on='id', how='left')
parts_copy = pd.merge(parts_copy, legos['sets'], left_on='set_num', right_on='set_num', how='left')
parts_copy.drop(['inventory_id', 'id'], axis=1, inplace=True)
names = ['part_num', 'part_name', 'part_cat_id', 'color_id', 'quantity', 'is_spare', 'version', 'set_num', 'set_name', 'year', 'theme_id', 'num_parts']
parts_copy.columns = names
parts_copy

,part_num,part_name,part_cat_id,color_id,quantity,is_spare,version,set_num,set_name,year,theme_id,num_parts
0,0687b1,Set 0687 Activity Booklet 1,17,9999.0,1.0,f,1.0,970687-1,Team Challenge Upgrade Kit,2002.0,524.0,95.0
1,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1,2.0,1.0,f,1.0,080-1,Basic Building Set with Train,1967.0,366.0,710.0
2,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1,2.0,1.0,f,1.0,080-1,Basic Building Set with Train,1967.0,366.0,710.0
3,0903,Baseplate 16 x 24 with Set 080 Red House Print,1,2.0,1.0,f,1.0,080-1,Basic Building Set with Train,1967.0,366.0,710.0
4,0904,Baseplate 16 x 24 with Set 080 Large White Hou...,1,2.0,1.0,f,1.0,080-1,Basic Building Set with Train,1967.0,366.0,710.0
5,1,Homemaker Bookcase 2 x 4 x 4,7,15.0,1.0,f,1.0,292-1,Kitchen Sink and Cupboards,1973.0,405.0,51.0
6,1,Homemaker Bookcase 2 x 4 x 4,7,15.0,2.0,f,1.0,295-1,Secretary's Desk,1974.0,405.0,80.0
7,1,Homemaker Bookcase 2 x 4 x 4,7,4.0,3.0,f,1.0,294-1,Wall Unit,1974.0,405.0,58.0
8,1,Homemaker Bookcase 2 x 4 x 4,7,15.0,3.0,f,1.0,260-3,Complete Living Room Set,1971.0,405.0,177.0
9,1,Homemaker Bookcase 2 x 4 x 4,7,14.0,3.0,f,1.0,263-1,Kitchen Set,1974.0,405.0,173.0


## 5. Create a copy of the set_themes table created earlier. Modify it to create a new table indicating how many transparent parts there are for each set name

In [170]:
set_themes_copy = set_themes.copy()
set_themes_copy = pd.merge(set_themes_copy, legos['inventories'], left_on='set_num', right_on='set_num', how='inner')
set_themes_copy = pd.merge(set_themes_copy, legos['inventory_parts'], left_on='id', right_on='inventory_id', how='inner')
set_themes_copy = pd.merge(set_themes_copy, legos['colors'], left_on='color_id', right_on='id', how='inner')
set_themes_copy['is_trans'] = set_themes_copy['is_trans'].apply(lambda x: 1 if x == 't' else 0)
set_themes_copy = set_themes_copy.groupby(['set_num', 'set_name', 'theme_name', 'parent_theme_id'])['quantity', 'is_trans'].sum()
set_themes_copy.reset_index(inplace=True)
names = ['set_num', 'set_name', 'theme_name', 'parent_theme_id', 'quantity', 'transparent_quantity']
set_themes_copy.columns = names
set_themes_copy

,set_num,set_name,theme_name,parent_theme_id,quantity,transparent_quantity
0,00-1,Weetabix Castle,Castle,411.0,471,0
1,00-2,Weetabix Promotional House 1,Building,411.0,147,2
2,00-3,Weetabix Promotional House 2,Building,411.0,149,0
3,00-4,Weetabix Promotional Windmill,Building,411.0,126,0
4,0011-2,Town Mini-Figures,Supplemental,67.0,12,0
5,0012-1,Space Mini-Figures,Supplemental,126.0,12,0
6,0013-1,Space Mini-Figures,Supplemental,126.0,12,0
7,0014-1,Space Mini-Figures,Supplemental,126.0,12,0
8,0015-1,Space Mini-Figures,Supplemental,126.0,18,0
9,005-1,Basic Building Set in Cardboard,Basic Set,365.0,35,0
